In [129]:
#!pip install pygame

In [130]:
import pygame
#from ball import Ball
#from cart import Cart
#from agent import Agent

In [131]:
window_size = 500, 500

In [132]:
import random
import math
#Класс мяча
class Ball:
    def __init__(self, surface, position):
        self.radius = 20 
        self.surface = surface
        self.position = position
        self.speed = 4
        
        angle = random.randint(-45, 45) + random.randint(0, 1) * 180
        
        self.velocity_x = self.speed * math.cos(angle * math.pi / 180)
        self.velocity_y = self.speed * math.sin(angle * math.pi / 180)
        
        self.color = (255,0,0) #Красный

    def draw(self):  
        pygame.draw.circle(self.surface, 
                           self.color, 
                           (int(self.position[0]), int(self.position[1])), 
                           self.radius)

    def set_Position(self, position):
        self.position = position
        
        angle = random.randint(-45, 45) + random.randint(0, 1) * 180
        
        self.velocity_x = self.speed * math.cos(angle * math.pi / 180)
        self.velocity_y = self.speed * math.sin(angle * math.pi / 180)

In [133]:
#Класс тележки
class Cart:
    def __init__(self, surface, position):
        self.velocity = 5
        self.size = [10, 50] 
        self.position = position
        self.surface = surface
        self.color = (0,255,0) 
    
    def draw(self): 
        pygame.draw.rect(self.surface, 
                         self.color, 
                         pygame.Rect(self.position[0],  
                                     self.position[1], 
                                     self.size[0], 
                                     self.size[1]))
    def set_Position(self, position):
        self.position = position

    def move_up(self):
        if self.position[1] <= window_size[1] - self.size[1]:
            self.position[1] += self.velocity
    def move_down(self):
        if self.position[1] > 0:
            self.position[1] -= self.velocity

In [134]:
#Класс вирутального игрока
class Agent():
    def __init__(self, cart):
        self.cart = cart
        self.last = self.cart.move_up
        self.n = -1
    #Алгоритм случайной игры    
    def auto_play(self, ball, cart):
        if self.n > 0:
            self.n -= 1
        else:
            self.n = 20
            if ball.position[1] > cart.position[1]:
                self.last = random.choice((self.cart.move_up, self.cart.move_down, self.cart.move_up, self.cart.move_up))
            else:
                self.last = random.choice((self.cart.move_up, self.cart.move_down, self.cart.move_down, self.cart.move_down))
        self.last()                          

In [135]:
class Game():
    def __init__(self):
        pygame.init()
        self.window = pygame.display.set_mode(window_size)
        pygame.display.set_caption('Ping Pong')

        #Создаю тележки
        self.cart_1 = Cart(self.window, [0, 250]) 
        self.cart_2 = Cart(self.window, [490, 250])
        #Создаю мяч
        self.ball = Ball(self.window, [250, 250])

        #Создаю агента
        self.agent = Agent(self.cart_2)

        #Рисую мяч и тележки
        self.cart_1.draw()
        self.cart_2.draw()
        self.ball.draw()
        pygame.display.update()
        
    # Функция получения текущей ситуации на игровом поле
    def get_obs(self):
        return (self.cart_1.position, self.cart_2.position, self.ball.position, self.ball.direction) 
    
    def game_cicle(self):
        game = True
        #Цикл игры
        while game:
            #Чтобы не так быстро
            pygame.time.delay(10)
            #Обработка событий
            for event in pygame.event.get():
                #Выход
                if event.type == pygame.QUIT:
                    game = False
                    pygame.quit()

            #Какие кнопки нажаты
            keys = pygame.key.get_pressed()
            #Если нажата вверх
            if keys[pygame.K_UP]:
                self.cart_1.move_down()
            #Если нажата вниз
            if keys[pygame.K_DOWN]:
                self.cart_1.move_up()
            #Если нажата W
            if keys[pygame.K_w]:
                self.cart_2.move_down()
            #Если нажата S
            if keys[pygame.K_s]:
                self.cart_2.move_up()
            if keys[pygame.K_ESCAPE]:
                game = False
                pygame.quit()
        
            #Если шар касается тележки #Поменяй шар на квадрат, иначе придется тут переписать      
            if (self.cart_2.position[0] - self.ball.position[0] - self.ball.radius <= 0 and
                self.cart_2.position[0] + self.cart_2.size[0]/2 - self.ball.position[0] - self.ball.radius > 0 and 
                self.cart_2.position[1] - self.ball.position[1] < self.ball.radius / 1.5 and
                self.ball.position[1] - self.cart_2.position[1] - self.cart_2.size[1] < self.ball.radius / 1.5 or
                self.cart_1.position[0] + self.cart_1.size[0] - self.ball.position[0] + self.ball.radius >= 0 and
                self.cart_1.position[0] + self.cart_1.size[0]/2 - self.ball.position[0] + self.ball.radius < 0 and
                self.cart_1.position[1] - self.ball.position[1] < self.ball.radius / 1.5 and
                self.ball.position[1] - self.cart_1.position[1] - self.cart_1.size[1] < self.ball.radius / 1.5):
                self.ball.velocity_x = -1 * self.ball.velocity_x
            
            #Отражение
            if (self.ball.position[1] - self.ball.radius <= 0 or 
                self.ball.position[1] + self.ball.radius >= window_size[1]):
                # изменить направление скорости по y
                self.ball.velocity_y = -1 * self.ball.velocity_y 
        
        
            #Если шар касается сторон экрана для завершения
            if (self.ball.position[0] <= 0 or 
                self.ball.position[0] >= window_size[0]):
                # изменить направление скорости по y
                self.ball.velocity_y = -1 * self.ball.velocity_y 
                game = False

            #Переместить шар
            self.ball.position[0] += self.ball.velocity_x
            self.ball.position[1] += self.ball.velocity_y 

            self.agent.auto_play(self.ball, self.cart_2)

            #Очистка экрана
            self.window.fill((0,0,0))
            #Заново рисуем тележку
            self.cart_1.draw()   
            self.cart_2.draw()

            #Заново рисуем шар
            self.ball.draw()
            #Обновляем экран
            pygame.display.update()
    
    
    def start(self):
        while True:
            self.game_cicle()
            self.cart_1.set_Position([0, 250])    
            self.cart_2.set_Position([490, 250])
            self.ball.set_Position([250,250])
        pygame.quit()

In [136]:
game = Game()
game.start()

error: display Surface quit